In [6]:
import h5py
import numpy as np 
import matplotlib.pyplot as plt 
import matplotlib.image as mpimg
import matplotlib.patches as mpatches
import tensorflow
import keras
import os
import cv2
from PIL import Image
keras.utils.generic_utils = keras.utils
from keras.optimizers import SGD, Adam
import random
from keras.models import Sequential, Model, load_model
# from keras.layers import Dense, Dropout, Activation, Flatten,Reshape,Input
# from keras.layers import Conv2D, MaxPooling2D, UpSampling2D,concatenate
# from keras.layers.convolutional.conv2d_transpose import Conv2DTranspose
import matplotlib

In [7]:
patch_size=256
img_height = 3000
img_width = 4000
n_classes= 10
train_grid_size = 256
height = train_grid_size
width = train_grid_size
resize_dim = (2*train_grid_size, 2*train_grid_size)
h_n = int(resize_dim[0]/height)
w_n = int(resize_dim[0]/width)

In [8]:
wd = os.getcwd()
wd

'/content'

In [9]:
training_img_path= wd + '/drive/MyDrive/Floodnet/train/train-org-img'

In [10]:
images= os.listdir(training_img_path)

In [11]:
train_floodnet_images =[]
for img_path in images:
  img = cv2.imread("/".join((training_img_path, img_path)),1)
  img = img[0:img_height, 0:img_width,:]
  img = cv2.resize(img, resize_dim)
  i = 0
  j = 0
  y = 0

  for i in range(h_n):
        x=0
        for j in range(w_n):
            img_crop = img[y:(y+height), x:(x+width), :]
            train_floodnet_images.append(img_crop)
            x+=width
        y+=height
train_floodnet_images = np.array(train_floodnet_images)


In [12]:
training_mask_path =os.getcwd() + '/drive/MyDrive/Floodnet/train/train-label-img'

In [13]:
train_masks= os.listdir(training_mask_path)

In [14]:
train_flooded_mask=[]
for mask_path in train_masks:
  mask = cv2.imread("/".join((training_mask_path, mask_path)),0)
  mask = mask[0:img_height, 0:img_width]
  mask = cv2.resize(mask, resize_dim)
  i = 0
  j = 0
  y = 0

  for i in range(h_n):
        x=0
        for j in range(w_n):
            mask_crop = mask[y:(y+height), x:(x+width)]
            train_flooded_mask.append(mask_crop)
            x+=width
        y+=height
train_flooded_mask = np.array(train_flooded_mask)

In [15]:
test_img_path = os.getcwd()+ '/drive/MyDrive/Floodnet/test/test-org-img'

In [16]:
test_images = os.listdir(test_img_path)

In [17]:
test_flooded_images =[]
for img_path in test_images:
  img = cv2.imread("/".join((test_img_path, img_path)),1)
  img = img[0:img_height, 0:img_width,:]
  img = cv2.resize(img, resize_dim)
  i = 0
  j = 0
  y = 0

  for i in range(h_n):
        x=0
        for j in range(w_n):
            img_crop = img[y:(y+height), x:(x+width), :]
            test_flooded_images.append(img_crop)
            x+=width
        y+=height
test_flooded_images = np.array(test_flooded_images)


In [18]:
test_mask_path = os.getcwd()+ '/drive/MyDrive/Floodnet/test/test-label-img'

In [19]:
test_masks = os.listdir(test_mask_path)

In [ ]:
test_flooded_mask=[]
for mask_path in test_masks:
  mask = cv2.imread("/".join((test_mask_path, mask_path)),0)
  mask = mask[0:img_height, 0:img_width]
  mask = cv2.resize(mask, resize_dim)
  i = 0
  j = 0
  y = 0

  for i in range(h_n):
        x=0
        for j in range(w_n):
            mask_crop = mask[y:(y+height), x:(x+width)]
            test_flooded_mask.append(mask_crop)
            x+=width
        y+=height
test_flooded_mask = np.array(test_flooded_mask)

In [ ]:
val_img_path = os.getcwd()+'/drive/MyDrive/Floodnet/val/val-org-img'

In [ ]:
val_images=  os.listdir(val_img_path)

In [ ]:
val_flooded_images =[]
for img_path in val_images:
  img = cv2.imread("/".join((val_img_path, img_path)),1)
  img = img[0:img_height, 0:img_width,:]
  img = cv2.resize(img, resize_dim)
  i = 0
  j = 0
  y = 0

  for i in range(h_n):
        x=0
        for j in range(w_n):
            img_crop = img[y:(y+height), x:(x+width), :]
            val_flooded_images.append(img_crop)
            x+=width
        y+=height
val_flooded_images = np.array(val_flooded_images)


In [ ]:
val_mask_dir = wd+ "/drive/MyDrive/Floodnet/val/val-label-img"

In [ ]:
val_flooded_mask = []
mask_3= os.listdir(val_mask_dir)
for mask_path in mask_3:
  mask = cv2.imread("/".join((val_mask_dir,mask_path)),0)
  mask = mask[0:img_height,0:img_width]
  mask = cv2.resize(mask, resize_dim)
  i=0
  j=0
  y=0
  for i in range(h_n):
    x=0
    for j in range(w_n):
      mask_crop = mask[y:(y+height),x:(x+width)]
      val_flooded_mask.append(mask_crop)
      x+=width
    y+=height
val_flooded_mask = np.array(val_flooded_mask)

In [ ]:
def prediction(model, image, patch_size):
  segm_img = np.zeros(image.shape[:2])
  patch_num=1
  for i in range(0, image.shape[0],256):
    for j in range(0, image.shape[1],256):
      single_patch = image[i:i+patch_size, j:j+patch_size]
      single_patch_norm = np.expand_dims(normalize(np.array(single_patch),axis=1),2)
      single_patch_shape = single_patch_norm.shape[:2]
      single_patch_input = np.expand_dims(single_patch_norm,0)
      single_patch_prediction = (model.predict(single_patch_input)[0,:,:,0]>0.5).astype(np.uint8)
      segm_img[i:i+single_patch_shape[0],j:j+single_patch_shape[1]] += cv2.resize(single_patch_prediction, single_patch_shape[::-1])

      print("Finished Processing Patch Number", patch_num, "At Position", i,j)

      patch_num +=1 

      return segm_img

In [ ]:
# print('Trainset Images:',train_flooded_images.shape)
# print('Trainset Masks:', train_flooded_mask.shape)
# print('Trainset Mask labels:', np.unique(train_flooded_mask))
# print('------------------------------------------------')
# print('Testset Images:',test_flooded_images.shape)
# print('Testset Masks:', test_flooded_mask.shape)
# print('Testset Mask labels:', np.unique(test_flooded_mask))
# print('------------------------------------------------')
# print('Validationset Images:',val_flooded_images.shape)
# print('Validationset Masks:', val_flooded_mask.shape)
# print('Validationset Mask labels:', np.unique(val_flooded_mask))

In [ ]:
Unet_tuned = load_model('/content/drive/MyDrive/unet_tuned_model.h5')

In [ ]:
Unet_untuned = load_model('/content/drive/MyDrive/model_untuned.hdf5')

In [ ]:
pspnet = load_model('/content/drive/MyDrive/psp_model_resnet50.h5')

In [ ]:
# def evaluate_random_image(model, test_images, test_masks):
#     # choose a random test image
#     random_index = np.random.choice(len(test_images))
#     test_image = test_images[random_index]
    
#     #predictions on the test image
#     test_image = np.expand_dims(test_image, axis=0)
#     predictions = model.predict(test_image)
    
#     #predicted mask
#     class_id = predictions.argmax(axis=-1)

#     #ground truth 
#     ground = test_masks[random_index]
#     ground = np.squeeze(ground)
#     # create a colormap for 10 classes
#     cmap = matplotlib.cm.get_cmap('tab10')
    
#     # display the test image, predicted mask, ground truth
#     plt.figure(figsize=(15,8))
#     plt.subplot(1,3,1)
#     plt.imshow(test_image[0])
#     plt.title('Test Image')
#     plt.axis('off')

#     plt.subplot(1,3,2)
#     plt.imshow(ground)
#     plt.title('Ground Truth')
#     plt.axis('off')

#     plt.subplot(1,3,3)
#     plt.imshow(class_id[0])
#     plt.title('Predicted Mask')
#     plt.axis('off')

#     # class_labels = ['Building Non-flooded', 'Building Flooded', 'Road Non-Flooded', 'Road', 'Class 5', 
#     #                 'Tree', 'Class 7', 'Car', 'Pool', 'Background']
#     # patches = [mpatches.Patch(color=cmap(i), label=class_labels[i]) for i in range(10)]
#     # plt.legend(handles=patches, bbox_to_anchor=(1.05, 1), loc='upper left')
    
#     plt.show()

In [ ]:
# evaluate_random_image(Unet_tuned, test_flooded_images, test_flooded_mask)

In [ ]:
# evaluate_random_image(Unet_untuned, test_flooded_images, test_flooded_mask)

In [ ]:
# ### Applying Trained Models on Large Image 
# large_image = cv2.imread('/content/drive/MyDrive/Floodnet/train/train-org-img/10687_lab.png',0)

In [ ]:
# large_image_norm = np.expand_dims(normalize(np.array(large_image),axis=1),2)

In [ ]:
# large_image_input = np.expand_dims(large_image_norm, 0)

In [ ]:
# segmented_image = prediction(Unet_untuned,large_image,patch_size)

# plt.figure(figsize=(10,8)
# plt.subplot(1,2,1)
# plt.title('Large Image')
# plt.imshow(large_image)

# plt.subplot(1,2,2)
# plt.title('Prediction on Large Image')
# plt.imshow(segmented_image)